In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_excel('C:\\Users\\BIT\\Downloads\\Complain.xlsx')

In [3]:
df.head()

,Domain,Complaint/ Opinion,Complaint Label,Complaint_Cause,Severity level,Sentiment,Emotion
0,Transaction Fail,Can someone tell me if transaction fail betwee...,1,transaction fail between merchant and Bank due...,No explicit reproach,negative,sadness
1,Transaction Fail,@LICIndiaForever @Paytmcare @Paytm As part of ...,1,balance deducted but payment status shows fail,No explicit reproach,negative,sadness
2,Transaction Fail,@ICICIBank_Care This is something very serious...,1,not able to use imobile app,No explicit reproach,negative,anger
3,Transaction Fail,@RBI @ICICIBank_Care @ICICIBank @CRED_support ...,1,payment seems a fraud transaction,Blame,negative,anger
4,Transaction Fail,Fraud transaction through CRED for SBI C-Card ...,1,Fraud transaction through CRED,Accusation,negative,surprise


In [4]:
df['Sentiment'].unique()

array(['negative', 'neutral', 'positive'], dtype=object)

In [5]:
df=df[['Complaint/ Opinion' , 'Sentiment']]

In [6]:
df.head()

,Complaint/ Opinion,Sentiment
0,Can someone tell me if transaction fail betwee...,negative
1,@LICIndiaForever @Paytmcare @Paytm As part of ...,negative
2,@ICICIBank_Care This is something very serious...,negative
3,@RBI @ICICIBank_Care @ICICIBank @CRED_support ...,negative
4,Fraud transaction through CRED for SBI C-Card ...,negative


In [7]:
X=df['Complaint/ Opinion']

In [8]:
y=df['Sentiment']

In [9]:
texts =X.astype(str).tolist()

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [12]:
def get_bert_embeddings(text_list, tokenizer, model, device, max_len=128):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list, desc="Encoding with BERT"):
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=max_len
            ).to(device)
            
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state  
            cls_embedding = last_hidden_state[:, 0, :].cpu().numpy().flatten()

            embeddings.append(cls_embedding)
    return np.array(embeddings)

In [13]:
bert_embeddings = get_bert_embeddings(texts, tokenizer, model, device)


ncoding with BERT: 100%|███████████████████████████████████████████████████████████████████████| 6280/6280 [19:13<00:00,  5.44it/s]

In [14]:
df["bert_embedding"] = list(bert_embeddings)

In [28]:
from transformers import BertTokenizer, BertModel
import torch

# Load pretrained BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

# Save BERT model and tokenizer
save_dir = "saved_bert_model"

tokenizer.save_pretrained(save_dir)
bert_model.save_pretrained(save_dir)   # ✅ use bert_model here, not model

print(f"✅ Saved BERT model and tokenizer to: {save_dir}")


✅ Saved BERT model and tokenizer to: saved_bert_model


In [15]:
bert_embeddings.shape

(6280, 768)

In [16]:
X=df["bert_embedding"]

In [17]:
y=df["Sentiment"]

In [25]:
y.replace({'negative':0 , 'positive':1 , 'neutral':2},inplace=True)

C:\Users\BIT\AppData\Local\Temp\ipykernel_20312\2885081918.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y.replace({'negative':0 , 'positive':1 , 'neutral':2},inplace=True)


In [27]:
y.value_counts()

Sentiment
0    3567
1    1462
2    1251
Name: count, dtype: int64

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Prepare data
X = np.vstack(df["bert_embedding"].values)
y = df["Sentiment"].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

# Define input/output dimensions
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model
model = nn.Sequential(
    nn.Linear(input_dim, 512),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(128, output_dim)
)
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train model
epochs = 20
batch_size = 64

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.size()[0])
    total_loss = 0

    for i in range(0, X_train.size()[0], batch_size):
        idx = permutation[i:i+batch_size]
        batch_X, batch_y = X_train[idx].to(device), y_train[idx].to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] Avg Loss: {total_loss:.4f}")

# Save model
torch.save(model.state_dict(), "best_model.pth")
print("\n🎯 Model saved as 'best_model.pth'")

# Evaluate model
model.eval()
with torch.no_grad():
    preds = model(X_test.to(device))
    predicted = torch.argmax(preds, dim=1).cpu().numpy()
    acc = accuracy_score(y_test, predicted)

print(f"✅ Test Accuracy: {acc * 100:.2f}%")


Epoch [1/20] Avg Loss: 76.5668
Epoch [2/20] Avg Loss: 60.3641
Epoch [3/20] Avg Loss: 52.1208
Epoch [4/20] Avg Loss: 49.5468
Epoch [5/20] Avg Loss: 47.4389
Epoch [6/20] Avg Loss: 45.7759
Epoch [7/20] Avg Loss: 45.1001
Epoch [8/20] Avg Loss: 44.0340
Epoch [9/20] Avg Loss: 42.8999
Epoch [10/20] Avg Loss: 42.5117
Epoch [11/20] Avg Loss: 41.6756
Epoch [12/20] Avg Loss: 40.9581
Epoch [13/20] Avg Loss: 40.8050
Epoch [14/20] Avg Loss: 39.4343
Epoch [15/20] Avg Loss: 39.0686
Epoch [16/20] Avg Loss: 37.7873
Epoch [17/20] Avg Loss: 36.7045
Epoch [18/20] Avg Loss: 35.7551
Epoch [19/20] Avg Loss: 35.2264
Epoch [20/20] Avg Loss: 34.2394

🎯 Model saved as 'best_model.pth'
✅ Test Accuracy: 76.83%


##y   0-->negactive  , 1-->neutral , 2-->positive

In [30]:
df

,Complaint/ Opinion,Sentiment,bert_embedding
0,Can someone tell me if transaction fail betwee...,negative,"[0.015723862, -0.11892362, 0.2700489, 0.087013..."
1,@LICIndiaForever @Paytmcare @Paytm As part of ...,negative,"[-0.23529556, -0.038245983, 0.2574466, -0.4277..."
2,@ICICIBank_Care This is something very serious...,negative,"[0.051478382, -0.04059323, 0.49588287, -0.0398..."
3,@RBI @ICICIBank_Care @ICICIBank @CRED_support ...,negative,"[0.044812456, -0.103598826, 0.3338256, 0.03373..."
4,Fraud transaction through CRED for SBI C-Card ...,negative,"[-0.07003464, -0.4320778, 0.28209257, 0.039742..."
...,...,...,...
6275,If your implication is that policies to reduce...,negative,"[0.2980588, 0.2017999, -0.45861152, -0.0231025..."
6276,Banks being put on notice regarding climate ch...,positive,"[-0.012771929, 0.09655386, 0.06859196, 0.04846..."
6277,How Will the Stock Market Do in 2022? Expect a...,positive,"[-0.24371383, -0.019919544, -0.1082685, -0.118..."
6278,"Goodbye, punch bowl. We’re going to miss you. ...",negative,"[-0.027334321, -0.039972924, -0.01948164, -0.1..."
